In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../data'
group_dir_ = os.path.join(dir_, 'groups/low')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in df['tid'].unique():
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,3,2.544071,1
1,0,7,2.545822,1
2,0,18,2.152490,1
3,0,20,1.882396,1
4,0,31,2.410806,1


In [6]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
166,166,3.840909
7209,7209,3.295455
350,350,3.113636
3761,3761,3.045455
80,80,3.022727
1017,1017,2.840909
1753,1753,2.750000
428,428,2.727273
200,200,2.681818
3159,3159,2.659091


In [7]:
top_percent = 0.01
long_tail = df_pop[int(len(df_pop)*top_percent):]
long_tail = long_tail['tid'].unique()
len(long_tail), len(df_pop), len(df_pop)-len(long_tail)

(54420, 54969, 549)

In [8]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops = []
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j]
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])                    
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    print(avg_pops)

In [11]:
HitNOVs = []
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    HitNOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        
        nov = 0
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        nov += (1-pop_dict[tid])
        
        HitNOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
    HitNOVs.append(HitNOV)
    
for HitNOV in HitNOVs:
    print(HitNOV)


[0.03917030303030301, 0.03443250688705238, 0.02779306220095695, 0.024713286713286715, 0.02616818181818183, 0.023266233766233775, 0.02174512987012987]
[0.045226666666666596, 0.03787786960514237, 0.029740430622009578, 0.024899300699300693, 0.02636818181818183, 0.023474025974025977, 0.022147727272727274]
[0.05351878787878772, 0.04111202938475668, 0.032074162679425845, 0.0265062937062937, 0.02789545454545455, 0.02474025974025975, 0.021959415584415585]
[0.06573939393939376, 0.044853076216712626, 0.03547966507177036, 0.02704615384615385, 0.02817121212121212, 0.024228896103896114, 0.023233766233766225]
[0.07533151515151504, 0.04978787878787882, 0.039454545454545485, 0.02930909090909092, 0.029465151515151507, 0.025842532467532466, 0.023547077922077917]
[0.07895090909090906, 0.05324334251606979, 0.04308732057416274, 0.031048951048951053, 0.030853030303030302, 0.02846590909090909, 0.022612012987012983]
[0.07986666666666661, 0.05426997245179065, 0.04684569377990434, 0.03300699300699301, 0.034037

In [11]:
diversities = []
novelties = []
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_items = pd.DataFrame() 

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]   
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()

        top_n_items_g.append(top_n_items)   
    
    diversity = []
    for i in range(len(groups_n)):
        top_n_items = top_n_items_g[i]
        diversity.append(len(top_n_items)/len(df['tid'].unique()))
    diversities.append(diversity)

    novelty = []
    for i in range(len(groups_n)):
        top_n_items = top_n_items_g[i]
        novelty.append(len(top_n_items[top_n_items['tid'].isin(long_tail)])/len(df['tid'].unique()))
    novelties.append(novelty)
    
for diversity in diversities:
    print(diversity)
print()
for novelty in novelties:
    print(novelty)


[7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05]
[9.096035947534064e-05, 0.00010915243137040878, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05]
[0.00016372864705561317, 0.0001273445032654769, 0.0001273445032654769, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05]
[0.0003638414379013626, 0.0002001127908457494, 0.00016372864705561317, 0.00010915243137040878, 0.00010915243137040878, 0.00010915243137040878, 9.096035947534064e-05]
[0.0004729938692717714, 0.00034564936600629446, 0.0002546890065309538, 0.0002001127908457494, 0.0001819207189506813, 0.0001273445032654769, 0.00010915243137040878]
[0.0005821463006421802, 0.0004002255816914988, 0.0004002255816914988, 0.00027288107842602196, 0.0002364969346358857, 0.00021830486274081756, 0.0001819207189506813]
[0.00067310666011

In [9]:
tid_pd = pd.DataFrame(data={'tid': df['tid'].unique()})
tid_pd = tid_pd.sort_values(by=['tid']).reset_index()
long_tail_list = tid_pd['tid'].isin(long_tail)
long_tail_list = list(long_tail_list)
long_tail_list.count(True)

54420

In [10]:
diversities = []
novelties = []
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    diversity = []
    novelty = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        hits[int(tid)] = True
        
        diversity.append(list(hits).count(True)/len(df['tid'].unique()))
        novelty.append(list(hits&long_tail_list).count(True)/len(df['tid'].unique()))
    diversities.append(diversity)
    novelties.append(novelty)
    
for diversity in diversities:
    print(diversity)
print()
for novelty in novelties:
    print(novelty)


[0.0009096035947534065, 0.0009096035947534065, 0.0008914115228583384, 0.0008732194509632703, 0.0009096035947534065, 0.0008186432352780658, 0.0008368353071731339]
[0.0013825974640251778, 0.001127908457494224, 0.0010187560261238151, 0.0009459877385435427, 0.0009641798104386109, 0.0008732194509632703, 0.0008732194509632703]
[0.0018555913332969493, 0.001400789535920246, 0.0012006767450744966, 0.0010369480980188834, 0.0010369480980188834, 0.0009641798104386109, 0.0008732194509632703]
[0.002637850424784879, 0.0016736706143462678, 0.001418981607815314, 0.0011461005293892921, 0.0011097163855991559, 0.0009641798104386109, 0.0009277956666484746]
[0.0034383015881678763, 0.0020011279084574942, 0.0017282468300314723, 0.001291637104549837, 0.0012188688169695646, 0.0011461005293892921, 0.0009823718823336789]
[0.003947679601229784, 0.0023467772744637885, 0.0020193199803525625, 0.0015281340391857228, 0.0013825974640251778, 0.0013462133202350416, 0.0010551401699139516]
[0.00427513689534101, 0.002710618